In [1]:
from functools import partial
import numpy as np
import PyQt5
import qcodes
from qcodes.data.io import DiskIO
from qcodes.data.data_set import DataSet


from core_tools.GUI.keysight_videomaps.liveplotting import liveplotting, set_data_saver
from core_tools.GUI.keysight_videomaps.data_saver.qcodes import QCodesDataSaver
from core_tools.GUI.keysight_videomaps.data_getter.scan_generator_OPX import fake_digitizer
from core_tools.GUI.qt_util import qt_init

from pulse_lib.base_pulse import pulselib

from core_tools.drivers.gates import gates as Gates
from core_tools.drivers.hardware.hardware import hardware as Hardware

from core_tools.GUI.virt_gate_matrix_qml.gui_controller import (
    virt_gate_matrix_GUI as VirtGateMatrixGUI,
)
from core_tools.GUI.param_viewer.param_viewer_GUI_main import (
    param_viewer as ParamViewerGUI,
)

# QDACII driver
from qcodes_contrib_drivers.drivers.QDevil import QDAC2
# OPX driver
from qualang_tools.external_frameworks.qcodes.opx_driver import OPX

station = qcodes.Station()

# host: router IP address, port: 11+ last 3 dig of OPX
#opx = OPX(config, name="OPX_alice", host="opx-sc-001", port='11253', close_other_machines=False)
# opx = OPX(config, name="OPX_alice", host="opx-sc-001", port='11253', close_other_machines=True)
# #Add the OPX instrument to the QCoDeS station
# station.add_component(opx)


/tmp/ipykernel_2013864/2571828664.py:5: QCoDeSDeprecationWarning: The qcodes.data module is deprecated. Use "qcodes_loop.data" as an alternative.
  from qcodes.data.io import DiskIO
/tmp/ipykernel_2013864/2571828664.py:5: QCoDeSDeprecationWarning: The qcodes.data.io module is deprecated. Use "qcodes_loop.data.io" as an alternative.
  from qcodes.data.io import DiskIO
/tmp/ipykernel_2013864/2571828664.py:6: QCoDeSDeprecationWarning: The qcodes.data.data_set module is deprecated. Use "qcodes_loop.data.data_set" as an alternative.
  from qcodes.data.data_set import DataSet
/local/work/masker/core_tools/core_tools/GUI/keysight_videomaps/data_saver/qcodes.py:4: QCoDeSDeprecationWarning: The qcodes.measure module is deprecated. Use "qcodes_loop.measure" as an alternative.
  from qcodes.measure import Measure


2024-02-19 12:48:42,530 - qm - INFO     - Starting session: 2972fc8c-c023-4f4d-a30a-253dd4b12cd1


In [1]:
#print(f'python version = {sys.version}')
from qm import QuantumMachinesManager
qmm = QuantumMachinesManager(host='opx-sc-001', cluster_name="Cluster_1") # IP address of router
qlst = qmm.list_open_quantum_machines()
print(f'{qlst=}')
for q in qlst:
    qm = qmm.get_qm(q)
    qm.close()


2024-02-19 14:54:00,211 - qm - INFO     - Starting session: f08c6911-9d16-4e8a-bef2-b1b5bbda266a
2024-02-19 14:54:01,349 - qm - INFO     - Performing health check
2024-02-19 14:54:01,354 - qm - INFO     - Health check passed
qlst=['qm-1708382845521']


In [5]:
import numpy as np

AWG_VirtGateMatrix = [
    # Vgs13    Vgs11    Vgs09    Vds    Vcm]
    [  1,        0,       0,      0,      1],   # HG13b
    [  0,        1,       0,      0,      1],   # HG11b
    [  0,        0,       1,      0,      1],   # HG09b
    [  0,        0,       0,    +0.5,     1],   # HG07b
    [  0,        0,       0,    -0.5,     1],   # HG12b
] 
m = AWG_VirtGateMatrix
minv = np.linalg.inv(AWG_VirtGateMatrix)
print(minv)
np.matmul(m, [[23,24],[45,46],[67,68],[10,11],[1000,1001]])

[[ 1.   0.   0.  -0.5 -0.5]
 [ 0.   1.   0.  -0.5 -0.5]
 [ 0.   0.   1.  -0.5 -0.5]
 [ 0.   0.   0.   1.  -1. ]
 [ 0.   0.   0.   0.5  0.5]]


array([[1023. , 1025. ],
       [1045. , 1047. ],
       [1067. , 1069. ],
       [1005. , 1006.5],
       [ 995. ,  995.5]])

In [10]:
np.array(m).reshape(25)

array([ 1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  1. ,  0. ,
        0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0.5,  1. ,  0. ,  0. ,
        0. , -0.5,  1. ])

In [5]:
x = ((1,2,3), (4,5,6))
y = 0.123 + x[1][2]
y

6.123

In [13]:
x = ('a','b','c')

x.index('d')

ValueError: tuple.index(x): x not in tuple

In [15]:
import numpy as np
np.linspace(-10.3, +3.7, 11)

array([-10.3,  -8.9,  -7.5,  -6.1,  -4.7,  -3.3,  -1.9,  -0.5,   0.9,
         2.3,   3.7])

In [17]:
np.isclose( [0.1,0.2], [0.1,0.2,0,3]).all()

ValueError: operands could not be broadcast together with shapes (2,) (4,) 

In [21]:
lst = np.array(((1,2,3),(4,5,6)))
lst.reshape(6)

array([1, 2, 3, 4, 5, 6])

In [3]:
import numpy as np
lst = [1,2,3]
np.max(lst)

3

In [5]:

qdac2_addr = 'qdac-sc-001'
qdac2 = QDAC2.QDac2('QDAC', visalib='@py', address=f'TCPIP::{qdac2_addr}::5025::SOCKET')
# Add the QDAC2 instrument to the QCoDeS station
#station.add_component(qdac2)


Connected to: QDevil QDAC-II (serial:146, firmware:7-0.17.5) in 0.21s


In [87]:
c = qdac2.channel(1)
c.dc_constant_V()
c.read_current_A()
c.current_last_A()
c.measurement_range()
c.dc_mode()
c.output_range()

'HIGH'

In [43]:
for ch_num in range(1,25):
    chn = qdac2.channel(ch_num)
    chn.output_range('low')
    chn.measurement_range('low')
    chn.output_filter('dc')

In [34]:
for ch_num in range(1,25):
    #ch_num = 1
    chn = qdac2.channel(ch_num)
    chn.dc_constant_V(-1.3)

In [46]:
for ch_num in range(1,25):
    chn = qdac2.channel(ch_num)
    v = chn.dc_constant_V()
    i = chn.read_current_A()[0]
    measrng = chn.measurement_range()
    mode=chn.dc_mode()
    outrng=chn.output_range()
    outfltr=chn.output_filter()
    print( f'CH{ch_num:02}   {mode=:5} {outrng=:5} {measrng=} {outfltr=:6} {v:10.3f}V {i*1e9:15,.3f}nA') 


CH01   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V           0.190nA
CH02   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V          -1.017nA
CH03   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V          -1.196nA
CH04   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V          -2.892nA
CH05   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V          -2.552nA
CH06   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V           0.410nA
CH07   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V          -0.495nA
CH08   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V          -2.012nA
CH09   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V          -0.951nA
CH10   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V         -18.041nA
CH11   mode=FIX   outrng=LOW   measrng='LOW' outfltr=DC         -1.300V           0.479nA
CH12   mod

In [31]:
qdac2.reset()

In [ ]:
# for i in range(1, 1+24):

i = 10
qdac2.add_parameter(
            f'dac{i}',
            qcodes.DelegateParameter,
            source=qdac2.channel(i).dc_constant_V,
            label=f'dac{i}',
            unit='V',
            scale=0.001
        )


In [ ]:





qcodes.DelegateParameter('dac10',  qdac2.ch10.dc_constant_V, instrument=qdac2 )

In [ ]:
qdac2.ch15.dc_constant_V.set(0.1)
qdac2.ch15.dc_constant_V()

In [ ]:
qdac2.dac10.set(1000)


In [ ]:

qdac2.dac10()

In [6]:
qdac2.dac10()

9.7999811

In [3]:
hw = Hardware("hardware")


In [4]:

hw.dac_gate_map = {
    "VGS09_15": (0, 15),        
}


In [ ]:


# For each of the gates create a DelegateParameter to the underlying qdac2 channel function
for gat, gmap in hw.dac_gate_map.items():
    cnum = gmap[1]
    dac_n = f'dac{cnum:d}'
    fstr = f'ch{cnum:d}'
    func = getattr(qdac2, fstr)
    func = getattr(func, 'dc_constant_V' )
    qcodes.DelegateParameter(dac_n,  func )

#p = qc.Instrument._all_instruments
pppp = qcodes.Instrument
print(pppp)



In [3]:
pppp = qcodes.Instrument
print(pppp)

<class 'qcodes.instrument.instrument.Instrument'>


In [ ]:
qcodes.DelegateParameter('dac15',  gates. )

In [5]:
gates = Gates("gates", hardware=hw, dac_sources=[qdac2], dc_gain={})

In [9]:
gates.VGS09_15.set(0.11)

KeyError: ('dac15', 'setting gates_VGS09_15 to 0.11')